<img src="logo.png" alt="drawing" width="500"/>

## Python Data Engineering Challenge

Our system ingests search term data from Google Ads API into a PostgreSQL database, via an AWS S3 Data Lake.

Once ingested we score each search term with its Return On Ad Spend (ROAS).

```text
ROAS = conversion value / cost
```


### Task

Three CSVs have been given - campaigns.csv, adgroups.csv and search_terms.csv. 

First ingest these 3 CSVs into a database, ensure the data ingestion is idempotent. 

Secondly, the adgroup alias is in the format:

`Shift - Shopping - <country> - <campaign structure value> - <priority> - <random string> - <hash>`

We sometimes need to know the ROAS aggregated by `country` and/or by `priority`. 

Build something to allow for those aggregations to be queried easily.


### Submission

Please fork this repo to complete the challenge, once done email back link to your repo.

Good luck we are rooting for you!

In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse as up
import sql_server_creds

In [2]:
# Initialize postgresql server and engine
up.uses_netloc.append("postgres")
url = up.urlparse(sql_server_creds.url)
engine = create_engine(\
    f"postgresql+psycopg2://{url.username}:{url.password}@{url.hostname}/{url.path[1:]}?client_encoding=utf8")

def establish_connection():
    conn = psycopg2.connect(database=url.path[1:], user=url.username, password=url.password, host=url.hostname, port=url.port)
    cur = conn.cursor()
    print("Connection Established!")
    return conn, cur
    
def close_connection():
    conn.commit()
    conn.close()
    print("Changes Commited & Connection Closed!")

In [3]:
# Remove pandas truncation
pd.set_option('display.max_colwidth', None)

In [4]:
conn, cur = establish_connection()
cur.execute("DROP TABLE IF EXISTS adgroups_src_tbl;\
            CREATE TABLE adgroups_src_tbl (\
                ad_group_id BIGINT,\
                campaign_id BIGINT,\
                alias VARCHAR(1000),\
                status VARCHAR(50)\
           );")

close_connection()
pd.read_sql("select * from adgroups_src_tbl limit 10", engine)

Connection Established!
Changes Commited & Connection Closed!


,ad_group_id,campaign_id,alias,status


In [5]:
conn, cur = establish_connection()
with open(r'adgroups.csv', 'r') as f:
    next(f)
    #cur.copy_from(f, 'adgroups_src_tbl', sep=',') # cant deal with delimiter enclosed within quotes i.e ","
    cur.copy_expert("""COPY adgroups_src_tbl FROM STDIN WITH (FORMAT CSV)""",f)

close_connection()

Connection Established!
Changes Commited & Connection Closed!


In [6]:
conn, cur = establish_connection()
cur.execute("DROP TABLE IF EXISTS campaigns_src_tbl;\
            CREATE TABLE campaigns_src_tbl (\
                campaign_id BIGINT,\
                structure_value VARCHAR(500),\
                status VARCHAR(50)\
           );")

close_connection()
pd.read_sql("select * from campaigns_src_tbl limit 10", engine)

Connection Established!
Changes Commited & Connection Closed!


,campaign_id,structure_value,status


In [7]:
conn, cur = establish_connection()
with open(r'campaigns.csv', 'r') as f:
    next(f)
    cur.copy_expert("""COPY campaigns_src_tbl FROM STDIN WITH (FORMAT CSV)""",f)

close_connection()

Connection Established!
Changes Commited & Connection Closed!


In [8]:
conn, cur = establish_connection()
cur.execute("DROP TABLE IF EXISTS search_terms_src_tbl;\
            CREATE TABLE search_terms_src_tbl (\
                date DATE, ad_group_id BIGINT,\
                campaign_id BIGINT,\
                clicks INT, \
                cost FLOAT,\
                conversion_value FLOAT,\
                conversions INT,\
                search_term VARCHAR(2000)\
           );")

close_connection()
pd.read_sql("select * from search_terms_src_tbl limit 10", engine)

Connection Established!
Changes Commited & Connection Closed!


,date,ad_group_id,campaign_id,clicks,cost,conversion_value,conversions,search_term


In [9]:
conn, cur = establish_connection()
with open(r'search_terms.csv', 'r') as f:
    next(f)
    cur.copy_expert("""COPY search_terms_src_tbl FROM STDIN WITH (FORMAT CSV)""",f)

close_connection()

Connection Established!
Changes Commited & Connection Closed!


In [10]:
pd.read_sql("select * from adgroups_src_tbl limit 10", engine)

,ad_group_id,campaign_id,alias,status
0,66372665454,1578451881,Shift - Shopping - GB - venum - LOW - monkey-ack-robert-comet - 817ce4882dfc499886ca8670ccd5cbf9,REMOVED
1,84481260174,1578451584,Shift - Shopping - GB - ellesse - HIGH - oscar-gee-princess-mexico - d77d4e4c99a4462991dd51ae0051e039,ENABLED
2,59624654596,1578451386,Shift - Shopping - GB - Converse - HIGH - failed-five-cola-mockingbird - 7f9a1fbba2e14a4e95bb7375181f0c9b,REMOVED
3,59977520149,1578412457,Shift - Shopping - GB - Wilson - HIGH - vermont-oregon-oscar-uncle - 611936a2347d4da3b2fb4aabe8f8ff0a,REMOVED
4,102171970298,9872103720,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED
5,102171970298,9872103720,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED
6,58309319903,1578451386,Shift - Shopping - GB - Converse - HIGH - three-equal-berlin-iowa - 6ddac5adf5ac4851bd9ed4e3108fd22b,REMOVED
7,119487436958,1578451623,Shift - Shopping - GB - spalding - HIGH - louisiana-april-nine-east - 5cf1d60650ec4ef58a72137040d58b26,ENABLED
8,59617309655,1578451386,Shift - Shopping - GB - Converse - HIGH - summer-whiskey-princess-september - 38a70b420a5f42e4a30d5f42e1620e4c,REMOVED
9,102171970298,9872103720,Shift - Shopping - GB - wham-o - HIGH - hawaii-delta-zebra-oranges - 09371471d42848869401d5bea7d18656,REMOVED


In [11]:
pd.read_sql("select * from campaigns_src_tbl limit 10", engine)

,campaign_id,structure_value,status
0,1578451881,venum,ENABLED
1,1578451584,ellesse,ENABLED
2,1578451386,converse,ENABLED
3,1578412457,wilson,ENABLED
4,9872103720,wham-o,ENABLED
5,9872103720,wham-o,ENABLED
6,1578451386,converse,ENABLED
7,1578451623,spalding,ENABLED
8,1578451386,converse,ENABLED
9,9872103720,wham-o,ENABLED


In [12]:
pd.read_sql("select * from search_terms_src_tbl limit 10", engine)

,date,ad_group_id,campaign_id,clicks,cost,conversion_value,conversions,search_term
0,2019-05-22,66372665454,1578451881,2,0.28,0.0,0,venum spats
1,2020-04-16,84481260174,1578451584,1,0.05,0.0,0,camillaw
2,2020-01-14,59624654596,1578451386,1,0.09,0.0,0,dc comic converse
3,2020-04-26,59977520149,1578412457,1,0.05,0.0,0,a1030
4,2020-05-30,102171970298,9872103720,1,0.17,0.0,0,h20 go
5,2020-05-30,102171970298,9872103720,1,0.11,0.0,0,asda waterslide mats
6,2019-04-23,58309319903,1578451386,1,0.01,0.0,0,cheapest infants glitter converse ebay
7,2021-06-08,119487436958,1578451623,2,0.16,0.0,0,the best basketball
8,2019-02-26,59617309655,1578451386,1,0.06,0.0,0,iridescent converse white
9,2020-05-27,102171970298,9872103720,1,0.11,0.0,0,slide mat


Our system ingests search term data from Google Ads API into a PostgreSQL database, via an AWS S3 Data Lake.

Once ingested we score each search term with its Return On Ad Spend (ROAS).

```text
ROAS = conversion value / cost
```


### Task

Three CSVs have been given - campaigns.csv, adgroups.csv and search_terms.csv. 

First ingest these 3 CSVs into a database, ensure the data ingestion is idempotent. 

Secondly, the adgroup alias is in the format:

`Shift - Shopping - <country> - <campaign structure value> - <priority> - <random string> - <hash>`

We sometimes need to know the ROAS aggregated by `country` and/or by `priority`. 

Build something to allow for those aggregations to be queried easily.

In [13]:
# Combine tables
pd.read_sql("select a.ad_group_id, a.campaign_id, a.alias,\
            split_part(a.alias,' - ',3) country, split_part(a.alias,' - ',5) priority, a.status ad_status,\
            c.structure_value, c.status campaign_status,\
            s.date search_date, s.clicks, s.cost, s.conversion_value, s.conversions, s.search_term,\
            case when s.conversion_value = '0' or s.cost = '0' then 0 else s.conversion_value/s.cost end as ROAS\
            from (select distinct * from adgroups_src_tbl) a\
            left join (select distinct * from campaigns_src_tbl) c on a.campaign_id = c.campaign_id\
            left join (select distinct * from search_terms_src_tbl) s on a.ad_group_id = s.ad_group_id\
            limit 10", engine)

,ad_group_id,campaign_id,alias,country,priority,ad_status,structure_value,campaign_status,search_date,clicks,cost,conversion_value,conversions,search_term,roas
0,55063206410,1578451878,Shift - Shopping - GB - Under Armour - HIGH - virginia-early-kansas-july - c3df7f93eaa8473380b3441837495b48,GB,HIGH,REMOVED,under armour,ENABLED,2018-12-12,2,0.05,0.0,0,under armour sports bra,0.0
1,55063210970,1578451878,Shift - Shopping - GB - Under Armour - HIGH - six-friend-lithium-yankee - 3eb55e1dad384683942a34f1d44b781b,GB,HIGH,REMOVED,under armour,ENABLED,2019-03-22,1,0.04,0.0,0,191168364896,0.0
2,55063210970,1578451878,Shift - Shopping - GB - Under Armour - HIGH - six-friend-lithium-yankee - 3eb55e1dad384683942a34f1d44b781b,GB,HIGH,REMOVED,under armour,ENABLED,2020-05-17,1,0.03,0.0,0,women's pink under armour t shirt,0.0
3,55063212890,1578451878,Shift - Shopping - GB - Under Armour - HIGH - mississippi-dakota-colorado-arkansas - 11098a1f131b4166a42478d26a7fcad6,GB,HIGH,ENABLED,under armour,ENABLED,2020-09-28,1,0.02,0.0,0,under armour youth coldgear leggings,0.0
4,55063212890,1578451878,Shift - Shopping - GB - Under Armour - HIGH - mississippi-dakota-colorado-arkansas - 11098a1f131b4166a42478d26a7fcad6,GB,HIGH,ENABLED,under armour,ENABLED,2020-09-28,1,0.03,0.0,0,boys under armour coldgear,0.0
5,55067334650,1578451386,Shift - Shopping - GB - Converse - HIGH - delta-eleven-red-winner - 1d60a5ea0d004b7b916d387cc4ae0498,GB,HIGH,REMOVED,converse,ENABLED,2019-01-06,1,0.12,0.0,0,black converse joggers,0.0
6,55067334650,1578451386,Shift - Shopping - GB - Converse - HIGH - delta-eleven-red-winner - 1d60a5ea0d004b7b916d387cc4ae0498,GB,HIGH,REMOVED,converse,ENABLED,2019-02-02,1,0.07,0.0,0,converse all star tracksuit mens,0.0
7,55067335170,1578451386,Shift - Shopping - GB - Converse - HIGH - golf-december-cola-video - d7f8d1ab8e8d4172b857f83f377c389d,GB,HIGH,REMOVED,converse,ENABLED,2020-01-13,1,0.16,0.0,0,converse polo t shirts,0.0
8,55067336810,1578630361,Shift - Shopping - GB - Puma - HIGH - helium-mango-mirror-failed - 8735cb252c97477e81e3f31d5b7ac8a8,GB,HIGH,ENABLED,puma,ENABLED,2019-03-02,1,0.01,0.0,0,boys size 13 football boots,0.0
9,55067337770,1578630361,Shift - Shopping - GB - Puma - HIGH - berlin-oklahoma-fish-dakota - b2e76120511a41a9b27872fed1da4266,GB,HIGH,ENABLED,puma,ENABLED,2019-02-14,1,0.11,0.0,0,puma evopower 3 red and black,0.0


In [14]:
conn, cur = establish_connection()
cur.execute("DROP TABLE IF EXISTS combined_tbl;\
            CREATE TABLE combined_tbl as (\
                SELECT a.ad_group_id, a.campaign_id, a.alias,\
                split_part(a.alias,' - ',3) country, split_part(a.alias,' - ',5) priority, a.status ad_status,\
                c.structure_value, c.status campaign_status,\
                s.date search_date, s.clicks, s.cost, s.conversion_value, s.conversions, s.search_term,\
                CASE WHEN s.conversion_value = '0' OR s.cost = '0' THEN 0 ELSE s.conversion_value/s.cost END AS ROAS\
                FROM (SELECT DISTINCT * FROM adgroups_src_tbl) a\
                LEFT JOIN (SELECT DISTINCT * FROM campaigns_src_tbl) c ON a.campaign_id = c.campaign_id\
                LEFT JOIN (SELECT DISTINCT * FROM search_terms_src_tbl) s ON a.ad_group_id = s.ad_group_id\
           );")

close_connection()
pd.read_sql("select * from combined_tbl limit 10", engine)

Connection Established!
Changes Commited & Connection Closed!


,ad_group_id,campaign_id,alias,country,priority,ad_status,structure_value,campaign_status,search_date,clicks,cost,conversion_value,conversions,search_term,roas
0,55063206410,1578451878,Shift - Shopping - GB - Under Armour - HIGH - virginia-early-kansas-july - c3df7f93eaa8473380b3441837495b48,GB,HIGH,REMOVED,under armour,ENABLED,2018-12-12,2,0.05,0.0,0,under armour sports bra,0.0
1,55063210970,1578451878,Shift - Shopping - GB - Under Armour - HIGH - six-friend-lithium-yankee - 3eb55e1dad384683942a34f1d44b781b,GB,HIGH,REMOVED,under armour,ENABLED,2019-03-22,1,0.04,0.0,0,191168364896,0.0
2,55063210970,1578451878,Shift - Shopping - GB - Under Armour - HIGH - six-friend-lithium-yankee - 3eb55e1dad384683942a34f1d44b781b,GB,HIGH,REMOVED,under armour,ENABLED,2020-05-17,1,0.03,0.0,0,women's pink under armour t shirt,0.0
3,55063212890,1578451878,Shift - Shopping - GB - Under Armour - HIGH - mississippi-dakota-colorado-arkansas - 11098a1f131b4166a42478d26a7fcad6,GB,HIGH,ENABLED,under armour,ENABLED,2020-09-28,1,0.02,0.0,0,under armour youth coldgear leggings,0.0
4,55063212890,1578451878,Shift - Shopping - GB - Under Armour - HIGH - mississippi-dakota-colorado-arkansas - 11098a1f131b4166a42478d26a7fcad6,GB,HIGH,ENABLED,under armour,ENABLED,2020-09-28,1,0.03,0.0,0,boys under armour coldgear,0.0
5,55067334650,1578451386,Shift - Shopping - GB - Converse - HIGH - delta-eleven-red-winner - 1d60a5ea0d004b7b916d387cc4ae0498,GB,HIGH,REMOVED,converse,ENABLED,2019-01-06,1,0.12,0.0,0,black converse joggers,0.0
6,55067334650,1578451386,Shift - Shopping - GB - Converse - HIGH - delta-eleven-red-winner - 1d60a5ea0d004b7b916d387cc4ae0498,GB,HIGH,REMOVED,converse,ENABLED,2019-02-02,1,0.07,0.0,0,converse all star tracksuit mens,0.0
7,55067335170,1578451386,Shift - Shopping - GB - Converse - HIGH - golf-december-cola-video - d7f8d1ab8e8d4172b857f83f377c389d,GB,HIGH,REMOVED,converse,ENABLED,2020-01-13,1,0.16,0.0,0,converse polo t shirts,0.0
8,55067336810,1578630361,Shift - Shopping - GB - Puma - HIGH - helium-mango-mirror-failed - 8735cb252c97477e81e3f31d5b7ac8a8,GB,HIGH,ENABLED,puma,ENABLED,2019-03-02,1,0.01,0.0,0,boys size 13 football boots,0.0
9,55067337770,1578630361,Shift - Shopping - GB - Puma - HIGH - berlin-oklahoma-fish-dakota - b2e76120511a41a9b27872fed1da4266,GB,HIGH,ENABLED,puma,ENABLED,2019-03-01,2,0.45,0.0,0,puma evopower vigor 3 fg,0.0


In [15]:
# Query where roas is greater than 0
pd.read_sql("select * from combined_tbl where roas > 0 limit 10", engine)

,ad_group_id,campaign_id,alias,country,priority,ad_status,structure_value,campaign_status,search_date,clicks,cost,conversion_value,conversions,search_term,roas
0,55067386730,1578630361,Shift - Shopping - GB - Puma - HIGH - maryland-yankee-idaho-charlie - c115baeb6f1b46ce89a46d2107192f14,GB,HIGH,REMOVED,puma,ENABLED,2019-01-27,1,0.10,22.98,1,best green football boots kids,229.800000
1,55067402050,1578412016,Shift - Shopping - GB - Kickers - HIGH - uniform-ack-yankee-steak - 7c05a53bf6d04066876af3e2bb2591db,GB,HIGH,ENABLED,kickers,ENABLED,2019-04-16,1,0.03,24.98,1,rose gold kicker boots,832.666667
2,55067982370,1578630361,Shift - Shopping - GB - Puma - HIGH - yankee-kilo-mango-mirror - 8ec7982d71be4ef091a9269cb605f5ca,GB,HIGH,REMOVED,puma,ENABLED,2019-02-18,1,0.08,37.98,1,dortmund home top,474.750000
3,55068003730,1578411800,Shift - Shopping - GB - Nike - HIGH - black-april-ten-echo - 9ab4330b5fec4efaacc454e666bc16f5,GB,HIGH,REMOVED,nike,ENABLED,2020-05-21,1,0.01,15.98,1,kids nike sweat shorts,1598.000000
4,55068004690,1578411800,Shift - Shopping - GB - Nike - HIGH - colorado-tennessee-july-september - c1af6622db904fdfa46a8a296afb1040,GB,HIGH,REMOVED,nike,ENABLED,2019-03-30,1,0.01,12.98,1,nike jersey shorts,1298.000000
5,55070131090,1578629881,Shift - Shopping - GB - adidas - LOW - lithium-lactose-lemon-chicken - b7adaacb2f38482b8fe2d0ca8c0104af,GB,LOW,REMOVED,adidas,ENABLED,2018-11-18,1,0.06,15.98,1,adidas crib shoes,266.333333
6,55070262370,1578451158,Shift - Shopping - GB - Barcelona - MEDIUM - enemy-william-oscar-romeo - 31b2378b71df4d16b1270f31e2f18728,GB,MEDIUM,REMOVED,barcelona,ENABLED,2019-04-25,1,0.29,8.98,1,barcelona water bottle,30.965517
7,55070262370,1578451158,Shift - Shopping - GB - Barcelona - MEDIUM - enemy-william-oscar-romeo - 31b2378b71df4d16b1270f31e2f18728,GB,MEDIUM,REMOVED,barcelona,ENABLED,2018-11-30,1,0.06,8.98,1,barcelona water bottle,149.666667
8,55070262410,1578451161,Shift - Shopping - GB - Barcelona - HIGH - arizona-colorado-friend-magnesium - 80d9fdac39384c67a7944f2c08b090a8,GB,HIGH,REMOVED,barcelona,ENABLED,2019-02-21,1,0.11,8.98,1,barcelona fc drink cup,81.636364
9,55070263130,1578629887,Shift - Shopping - GB - adidas - HIGH - bakerloo-pip-happy-north - ea64a53a19184ee084d2ada2230a8433,GB,HIGH,REMOVED,adidas,ENABLED,2019-04-28,1,0.01,41.98,1,adidas havoc boxing boots kids,4198.000000


In [16]:
# Query where search_term is a number
pd.read_sql("select * from combined_tbl where search_term ~ '^[0-9\.]+$' limit 10", engine)

,ad_group_id,campaign_id,alias,country,priority,ad_status,structure_value,campaign_status,search_date,clicks,cost,conversion_value,conversions,search_term,roas
0,55063210970,1578451878,Shift - Shopping - GB - Under Armour - HIGH - six-friend-lithium-yankee - 3eb55e1dad384683942a34f1d44b781b,GB,HIGH,REMOVED,under armour,ENABLED,2019-03-22,1,0.04,0.0,0,191168364896,0.0
1,56395713021,1578411800,Shift - Shopping - GB - Nike - HIGH - one-april-twelve-september - 72bd3f546c3248119c3b3c79b4c33f89,GB,HIGH,ENABLED,nike,ENABLED,2019-10-07,1,0.01,0.0,0,885179927873,0.0
2,56397358741,1578411965,Shift - Shopping - GB - Mitre - HIGH - white-alabama-stream-oklahoma - a38a1fc294ce471c8f972fe284d4689b,GB,HIGH,REMOVED,mitre,ENABLED,2018-11-17,1,0.08,0.0,0,5037823888862,0.0
3,58339202137,1578451560,Shift - Shopping - GB - Crep Protect - HIGH - illinois-berlin-beer-pennsylvania - 70e61de9a41e47e9aa097aca466a0280,GB,HIGH,ENABLED,crep protect,ENABLED,2019-01-16,1,0.12,0.0,0,660902996780,0.0
4,58595477894,1578630127,Shift - Shopping - GB - Optimum - HIGH - fourteen-bluebird-wyoming-failed - 8e3f74887e0a4f229626307ad3c398c9,GB,HIGH,ENABLED,optimum,ENABLED,2019-02-12,1,0.01,0.0,0,5055469645274,0.0
5,58703949337,1578451104,Shift - Shopping - GB - Chelsea - HIGH - alaska-delta-florida-music - 669d6b93db3b45979b65853179b0b7e8,GB,HIGH,REMOVED,chelsea,ENABLED,2018-12-10,1,0.11,0.0,0,5037970056664,0.0
6,58760134079,1578411800,Shift - Shopping - GB - Nike - HIGH - violet-mike-green-mars - b89de12b5b154480bde8141767185ba0,GB,HIGH,ENABLED,nike,ENABLED,2019-01-16,1,0.01,0.0,0,887791043030,0.0
7,59026618749,1578411800,Shift - Shopping - GB - Nike - HIGH - jersey-fruit-minnesota-carpet - 1320306a7c3c475696bccf6ea6e51574,GB,HIGH,ENABLED,nike,ENABLED,2018-12-14,1,0.05,0.0,0,95,0.0
8,59899098896,1578411800,Shift - Shopping - GB - Nike - HIGH - ohio-kansas-cola-sierra - 712dd42a51764c04b13253f84f398231,GB,HIGH,REMOVED,nike,ENABLED,2019-07-06,1,0.03,0.0,0,91209472941,0.0
9,59933752739,1578411800,Shift - Shopping - GB - Nike - HIGH - happy-freddie-kilo-arizona - 3f377c2d49704d3bac0e47949a0d6bf4,GB,HIGH,REMOVED,nike,ENABLED,2019-06-16,1,0.04,0.0,0,683485,0.0


In [17]:
# Query where campaign_id is 1578451878
pd.read_sql("select * from combined_tbl where campaign_id = '1578451878' limit 10", engine)

,ad_group_id,campaign_id,alias,country,priority,ad_status,structure_value,campaign_status,search_date,clicks,cost,conversion_value,conversions,search_term,roas
0,55063206410,1578451878,Shift - Shopping - GB - Under Armour - HIGH - virginia-early-kansas-july - c3df7f93eaa8473380b3441837495b48,GB,HIGH,REMOVED,under armour,ENABLED,2018-12-12,2,0.05,0.0,0,under armour sports bra,0.0
1,55063210970,1578451878,Shift - Shopping - GB - Under Armour - HIGH - six-friend-lithium-yankee - 3eb55e1dad384683942a34f1d44b781b,GB,HIGH,REMOVED,under armour,ENABLED,2019-03-22,1,0.04,0.0,0,191168364896,0.0
2,55063210970,1578451878,Shift - Shopping - GB - Under Armour - HIGH - six-friend-lithium-yankee - 3eb55e1dad384683942a34f1d44b781b,GB,HIGH,REMOVED,under armour,ENABLED,2020-05-17,1,0.03,0.0,0,women's pink under armour t shirt,0.0
3,55063212890,1578451878,Shift - Shopping - GB - Under Armour - HIGH - mississippi-dakota-colorado-arkansas - 11098a1f131b4166a42478d26a7fcad6,GB,HIGH,ENABLED,under armour,ENABLED,2020-09-28,1,0.02,0.0,0,under armour youth coldgear leggings,0.0
4,55063212890,1578451878,Shift - Shopping - GB - Under Armour - HIGH - mississippi-dakota-colorado-arkansas - 11098a1f131b4166a42478d26a7fcad6,GB,HIGH,ENABLED,under armour,ENABLED,2020-09-28,1,0.03,0.0,0,boys under armour coldgear,0.0
5,55067406930,1578451878,Shift - Shopping - GB - Under Armour - HIGH - iowa-kansas-sixteen-eleven - c7af121c3baa48b6a3627e760d7efbb5,GB,HIGH,ENABLED,under armour,ENABLED,2018-10-25,1,0.02,0.0,0,underarmour recharge tights,0.0
6,55067406930,1578451878,Shift - Shopping - GB - Under Armour - HIGH - iowa-kansas-sixteen-eleven - c7af121c3baa48b6a3627e760d7efbb5,GB,HIGH,ENABLED,under armour,ENABLED,2018-11-16,1,0.02,0.0,0,mens leggings,0.0
7,55067891890,1578451878,Shift - Shopping - GB - Under Armour - HIGH - three-ohio-four-thirteen - ea21c3d78e54473f9b75ff5d4d659d6b,GB,HIGH,ENABLED,under armour,ENABLED,2019-03-24,1,0.02,0.0,0,under armour vanish mid sports bra,0.0
8,55067907970,1578451878,Shift - Shopping - GB - Under Armour - HIGH - video-hotel-wisconsin-georgia - cfbbce54a74c4bffb6d5a2394c3d6745,GB,HIGH,ENABLED,under armour,ENABLED,2020-07-08,1,0.07,0.0,0,under armour ultimate speed,0.0
9,55067907970,1578451878,Shift - Shopping - GB - Under Armour - HIGH - video-hotel-wisconsin-georgia - cfbbce54a74c4bffb6d5a2394c3d6745,GB,HIGH,ENABLED,under armour,ENABLED,2020-05-21,1,0.05,0.0,0,under armour ultimate speed mens trainers,0.0
